# LOGIN

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# NGROK

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

!ngrok authtoken 2nqM9Yg2gCLfmStA7mtFN084iB5_qhkL9nepVwtJtYjG4apm

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://5cab-34-19-122-212.ngrok-free.app" -> "http://localhost:5000"


# CODE

In [ ]:
import requests
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
from flask import Flask, request
import torch

app = Flask(__name__)

@app.route('/receive', methods=['POST'])
def receive_message():
    model_id = "google/paligemma-3b-mix-224"
    device = "cuda:0"
    dtype = torch.bfloat16
    data = request.json
    url = data['text']
    image = Image.open(requests.get(url, stream=True).raw)

    image = image.resize((224, 224))

    model = PaliGemmaForConditionalGeneration.from_pretrained(
      model_id,
      torch_dtype=dtype,
      device_map=device,
      revision="bfloat16",
    ).eval()
    processor = AutoProcessor.from_pretrained(model_id)
    model = model.to(device)
    prompt = data['prompt']
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    model_inputs = model_inputs.to(device)
    input_len = model_inputs["input_ids"].shape[-1]

    with torch.inference_mode():
      generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]
      decoded = processor.decode(generation, skip_special_tokens=True)

      return decoded, 200

if __name__ == '__main__':
    app.run(port=5000)




 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
